In [7]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks
IMG_SIZE = 32
LR = 1e-3
import pandas as pd

from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
import glob

In [40]:
path = r'D:\downloads\קבצים לעבודת גמר\extraFiles\Training\all data\sync/**/*.csv' 
hands_df =[]
for filename in glob.iglob(path,recursive=True):
    name = filename.split('/')[-1].split('.')[0].lower()
    hand_csv =pd.read_csv(filename)
    time_from =hand_csv.head(1)["Time"][0]
    hand_csv = hand_csv.drop(hand_csv[hand_csv.Time<Time_from+7].index)
    if 'alone' in name:
        hand_csv['Type']=0
    if 'sync' in name:
        hand_csv['Type']=1    
    if 'spontan' in name:
        hand_csv['Type']=2
    hands_df.append(hand_csv)
    df=pd.concat(hands_df)
    
      


In [47]:
path =  r'D:\downloads\קבצים לעבודת גמר\extraFiles\Training\all data\sync/**/*.csv' 
hands_df =[]
for filename in glob.iglob(path,recursive=True):
    name = filename.split('/')[-1].split('.')[0].lower()
    hand_csv =pd.read_csv(filename)
    time_from =hand_csv.head(1)["Time"][0]
    hand_csv = hand_csv.drop(hand_csv[hand_csv.Time<Time_from+7].index)
    if 'alone' in name:
        hand_csv['Type']=0
    if 'sync' in name:
        hand_csv['Type']=1    
    if 'spontan' in name:
        hand_csv['Type']=2
    hands_df.append(hand_csv)
    df=pd.concat(hands_df)

In [48]:
df
df

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,Type
422,788.4187,61729,left,2,-138.10520,249.0437,21.309240,-11.360700,-26.260100,-12.07120,...,-168.4162,180.8961,19.90572,-317.1709,-37.17384,39.49545,0.000000,0.843670,0.0,1
423,788.4187,61729,right,2,70.78098,279.7763,53.523280,-37.708780,28.761840,-98.75040,...,101.8301,212.7941,64.75998,272.9008,12.31499,89.44131,0.000000,0.000000,0.0,1
424,788.4520,61731,left,2,-138.38290,250.0482,20.923750,-8.339315,52.976450,-13.69866,...,-168.2721,181.6959,20.00260,-318.7131,-35.46490,36.58178,0.000000,0.903126,0.0,1
425,788.4520,61731,right,2,69.88145,281.5859,49.926010,-29.898320,59.757730,-118.88680,...,100.8418,214.6148,61.39282,273.4266,15.71526,88.21880,0.000000,0.000000,0.0,1
426,788.4854,61734,left,2,-138.23200,250.8598,20.484280,16.585670,-0.654439,-11.17161,...,-167.1507,182.1053,21.02546,-324.7052,-30.08764,35.75453,0.038828,0.997329,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,856.2631,67309,right,2,101.91230,212.0264,42.535680,79.958100,113.410300,190.77140,...,131.8590,149.6264,66.09853,283.6185,-58.03491,95.29868,0.000000,0.000000,0.0,1
4496,856.2958,67311,left,2,-81.57359,183.5054,-7.185198,-63.247310,121.752700,62.95970,...,-109.7250,130.7078,34.03360,-295.5957,-14.97817,140.02630,0.000000,0.000000,0.0,1
4497,856.2958,67311,right,2,103.84750,214.7484,47.070820,61.117730,95.135310,155.74530,...,134.0416,151.5924,68.17693,288.9688,-54.13820,94.23959,0.000000,0.000000,0.0,1
4498,856.3295,67314,left,2,-84.05124,187.8450,-5.172269,-57.172490,109.203600,58.48146,...,-109.5167,131.6489,33.32945,-282.9649,-31.35462,135.07610,0.000000,0.000000,0.0,1


In [50]:

# Set index on a Dataframe
df.set_index(" Hand Type", 
              inplace = True)
  
# Using the operator .loc[]
# to select multiple rows
df_lf_sync = df.loc[["left"]]
df_rg_sync = df.loc[["right"]] 

KeyError: "None of [' Hand Type'] are in the columns"

In [51]:
df_lf_sync

,Time,Frame ID,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,...,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,Type
Hand Type,,,,,,,,,,,,,,,,,,,,,
left,788.4187,61729,2,-138.10520,249.0437,21.309240,-11.360700,-26.260100,-12.07120,1.468756,...,-168.4162,180.8961,19.90572,-317.1709,-37.173840,39.49545,0.000000,0.843670,0.0,1
left,788.4520,61731,2,-138.38290,250.0482,20.923750,-8.339315,52.976450,-13.69866,1.458599,...,-168.2721,181.6959,20.00260,-318.7131,-35.464900,36.58178,0.000000,0.903126,0.0,1
left,788.4854,61734,2,-138.23200,250.8598,20.484280,16.585670,-0.654439,-11.17161,1.433956,...,-167.1507,182.1053,21.02546,-324.7052,-30.087640,35.75453,0.038828,0.997329,0.0,1
left,788.5206,61736,2,-138.63190,251.4699,20.923460,-18.990280,29.153950,19.88656,1.420910,...,-167.4190,182.7053,22.45415,-329.6457,-25.634150,40.98439,0.075072,1.042080,0.0,1
left,788.5522,61738,2,-139.15750,251.6453,21.478590,-14.590970,1.932955,17.94077,1.387609,...,-169.4030,183.6025,25.00736,-339.8019,-17.815180,46.48571,0.117173,1.106836,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
left,856.1965,67304,2,-77.62601,173.7391,-14.213500,-46.750000,99.509110,71.01207,0.629213,...,-107.8500,127.8215,32.85250,-318.7630,20.751860,138.01250,0.000000,0.000000,0.0,1
left,856.2291,67306,2,-78.43058,176.0170,-12.152660,-33.690260,98.243610,59.77054,0.667618,...,-108.0805,128.1729,33.40391,-314.6744,14.308990,139.99090,0.000000,0.000000,0.0,1
left,856.2631,67309,2,-79.84506,180.2403,-9.015577,-68.451200,136.375800,87.63277,0.725945,...,-109.0018,129.5246,33.89856,-304.9600,-0.548136,142.01090,0.000000,0.000000,0.0,1


In [52]:
df_rg_sync

,Time,Frame ID,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,...,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,Type
Hand Type,,,,,,,,,,,,,,,,,,,,,
right,788.4187,61729,2,70.78098,279.7763,53.52328,-37.708780,28.76184,-98.7504,1.276016,...,101.83010,212.7941,64.75998,272.9008,12.31499,89.44131,0.0,0.0,0.0,1
right,788.4520,61731,2,69.88145,281.5859,49.92601,-29.898320,59.75773,-118.8868,1.274227,...,100.84180,214.6148,61.39282,273.4266,15.71526,88.21880,0.0,0.0,0.0,1
right,788.4854,61734,2,69.35614,287.9706,44.20788,-1.634530,155.30050,-124.7679,1.263836,...,99.80907,220.9135,56.50512,271.0192,21.24614,86.27451,0.0,0.0,0.0,1
right,788.5206,61736,2,69.32423,292.0058,40.43280,-2.184077,142.06900,-127.4935,1.254064,...,99.39062,224.8885,53.39783,268.0019,23.20201,84.36508,0.0,0.0,0.0,1
right,788.5522,61738,2,69.48622,295.5573,36.56711,4.993362,121.04460,-134.0518,1.246857,...,99.02974,228.2917,50.04153,263.7600,23.51421,81.60546,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
right,856.1965,67304,2,97.75611,205.9629,30.04651,28.362980,77.44335,162.1428,0.909181,...,129.12610,148.7041,62.82404,300.6781,-33.58672,128.75430,0.0,0.0,0.0,1
right,856.2291,67306,2,98.93275,208.3153,34.85991,44.383930,71.36857,187.5408,0.983621,...,129.28680,148.1146,63.11218,285.6505,-53.66640,106.02620,0.0,0.0,0.0,1
right,856.2631,67309,2,101.91230,212.0264,42.53568,79.958100,113.41030,190.7714,1.065334,...,131.85900,149.6264,66.09853,283.6185,-58.03491,95.29868,0.0,0.0,0.0,1


In [62]:
df_rg_sync=df_rg_sync.drop([' Frame ID', 'Type',' # hands'], axis = 1)

In [63]:
df_rg_sync.columns

Index(['Time', ' Position X', ' Position Y', ' Position Z', ' Velocity X',
       ' Velocity Y', ' Velocity Z', ' Pitch', ' Roll', ' Yaw', ' Wrist Pos X',
       ' Wrist Pos Y', ' Wrist Pos Z', ' Elbow pos X', ' Elbow Pos Y',
       ' Elbow Pos Z', ' Grab Strenth', ' Grab Angle', ' Pinch Strength'],
      dtype='object')

In [64]:
df_sync = pd.merge(df_lf_sync,df_rg_sync, how = "inner", on ="Time")

In [65]:
df_sync

,Time,Frame ID,# hands,Position X_x,Position Y_x,Position Z_x,Velocity X_x,Velocity Y_x,Velocity Z_x,Pitch_x,...,Yaw_y,Wrist Pos X_y,Wrist Pos Y_y,Wrist Pos Z_y,Elbow pos X_y,Elbow Pos Y_y,Elbow Pos Z_y,Grab Strenth_y,Grab Angle_y,Pinch Strength_y
0,788.4187,61729,2,-138.10520,249.0437,21.309240,-11.360700,-26.260100,-12.07120,1.468756,...,-0.842252,101.83010,212.7941,64.75998,272.9008,12.31499,89.44131,0.0,0.0,0.0
1,788.4520,61731,2,-138.38290,250.0482,20.923750,-8.339315,52.976450,-13.69866,1.458599,...,-0.834117,100.84180,214.6148,61.39282,273.4266,15.71526,88.21880,0.0,0.0,0.0
2,788.4854,61734,2,-138.23200,250.8598,20.484280,16.585670,-0.654439,-11.17161,1.433956,...,-0.802327,99.80907,220.9135,56.50512,271.0192,21.24614,86.27451,0.0,0.0,0.0
3,788.5206,61736,2,-138.63190,251.4699,20.923460,-18.990280,29.153950,19.88656,1.420910,...,-0.778442,99.39062,224.8885,53.39783,268.0019,23.20201,84.36508,0.0,0.0,0.0
4,788.5522,61738,2,-139.15750,251.6453,21.478590,-14.590970,1.932955,17.94077,1.387609,...,-0.755912,99.02974,228.2917,50.04153,263.7600,23.51421,81.60546,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2034,856.1965,67304,2,-77.62601,173.7391,-14.213500,-46.750000,99.509110,71.01207,0.629213,...,-0.495629,129.12610,148.7041,62.82404,300.6781,-33.58672,128.75430,0.0,0.0,0.0
2035,856.2291,67306,2,-78.43058,176.0170,-12.152660,-33.690260,98.243610,59.77054,0.667618,...,-0.525644,129.28680,148.1146,63.11218,285.6505,-53.66640,106.02620,0.0,0.0,0.0
2036,856.2631,67309,2,-79.84506,180.2403,-9.015577,-68.451200,136.375800,87.63277,0.725945,...,-0.574317,131.85900,149.6264,66.09853,283.6185,-58.03491,95.29868,0.0,0.0,0.0
2037,856.2958,67311,2,-81.57359,183.5054,-7.185198,-63.247310,121.752700,62.95970,0.762549,...,-0.612753,134.04160,151.5924,68.17693,288.9688,-54.13820,94.23959,0.0,0.0,0.0


In [67]:
df_sync=df_sync.drop([' # hands'], axis = 1)

In [69]:
df_sync.head()

,Time,Frame ID,Position X_x,Position Y_x,Position Z_x,Velocity X_x,Velocity Y_x,Velocity Z_x,Pitch_x,Roll_x,...,Yaw_y,Wrist Pos X_y,Wrist Pos Y_y,Wrist Pos Z_y,Elbow pos X_y,Elbow Pos Y_y,Elbow Pos Z_y,Grab Strenth_y,Grab Angle_y,Pinch Strength_y
0,788.4187,61729,-138.1052,249.0437,21.30924,-11.360700,-26.260100,-12.07120,1.468756,1.241289,...,-0.842252,101.83010,212.7941,64.75998,272.9008,12.31499,89.44131,0.0,0.0,0.0
1,788.4520,61731,-138.3829,250.0482,20.92375,-8.339315,52.976450,-13.69866,1.458599,1.252178,...,-0.834117,100.84180,214.6148,61.39282,273.4266,15.71526,88.21880,0.0,0.0,0.0
2,788.4854,61734,-138.2320,250.8598,20.48428,16.585670,-0.654439,-11.17161,1.433956,1.280766,...,-0.802327,99.80907,220.9135,56.50512,271.0192,21.24614,86.27451,0.0,0.0,0.0
3,788.5206,61736,-138.6319,251.4699,20.92346,-18.990280,29.153950,19.88656,1.420910,1.289919,...,-0.778442,99.39062,224.8885,53.39783,268.0019,23.20201,84.36508,0.0,0.0,0.0
4,788.5522,61738,-139.1575,251.6453,21.47859,-14.590970,1.932955,17.94077,1.387609,1.281788,...,-0.755912,99.02974,228.2917,50.04153,263.7600,23.51421,81.60546,0.0,0.0,0.0
